# PEDS example: Reaction--diffusion surrogate using a diffusion equation coarse solver

This example of PEDS shows how to use the code from: *Pestourie, Raphaël, et al. "Physics-enhanced deep surrogates for PDEs."*  In this example notebook, we illustrate PEDS for the surrogate of the reaction--diffusion equation's flux where the approximate solver is a coarse diffusion equation's solver. For this example, the input is the side length of 16 holes in a material.

NB: since the approximate solver is the same for the reaction-diffusion and the diffusion surrogates, training the surrogate models with the 25 holes structures for the reaction-diffusion equation necessitates only to change the dataset from `X/y_fisher16_small.csv` to `X/y_fourier16_small.csv`.
    
## Note on implementation

In this notebook, we run the training on a single processor and J=1.

However, the implementation of this code is meant to be parallel on multiple processors, where the element of a batch is computed in parallel over J groups of CPUs (J is the number models in the ensemble). For example, with a julia file `train_PEDS.jl`containing functions calls similart to the code in the cells below, the command to run large-scale training on a cluster would be:
    
`mpiexec -n 320 julia train_PEDS.jl` 

In [1]:
#initialize functions

include("../src/PEDS.jl")
include("../src/fourier_solver.jl")

##training functionalities
function train_distributed!(comm, commModel, commLeader, mloglik, m, loss, ps, loader, opt, validation_fes; logging=false)

    train_lossmean = 0.
    k = 0
    for d in loader
        X,y = d #change for Fourier
        train_loss, back = Zygote.pullback(() -> loss(commModel, mloglik, m, X, y), ps)
        train_lossmean *= k
        k+=1
        train_lossmean += train_loss
        train_lossmean /= k

        gs = back(Float32(1.))
        if debug && isleader
            if isnan(train_loss)
                @show (model_color, train_loss)
            end
        end
        for x in ps
            gs[x][:] .= (sum_reduce(commModel, (gs[x][:]) ))
            if debug && isleader
                if any(isnan.(gs[x][:]))
                    @show (model_color, train_loss)
                    @show (length(x), length(findall(isnan,x)), length(findall(isnan,gs[x][:])))
                end

                if any(isinf.(gs[x][:]))
                    @show (model_color, train_loss)
                    @show (length(x), length(findall(isinf,x)), length(findall(isinf,gs[x][:])))
                end
            end
        end

        if debug && isleader
            for p_ in ps
                if any(isnan.(p_))
                    @show (model_color, "before update")
                end
            end
        end
        Flux.Optimise.update!(opt, ps, gs)
        if debug && isleader
            for p_ in ps
                if any(isnan.(p_))
                    @show (model_color, "after update")
                end
            end
        end
    end

    # logging && push!(validation_fes, dFE(comm, commModel, commLeader, m))
    #changing for fourier
    if logging
        if MPI.Comm_rank(comm) == 0
            @show train_lossmean
            push!(validation_fes, train_lossmean)
        end
        MPI.Barrier(comm)
        valid_lossMSE = lossvalid(comm, commModel, commLeader, mloglik, m)
        valid_lossFE = dFE_1d(comm, commModel, commLeader, m)
        if MPI.Comm_rank(comm) == 0
            push!(validation_fes, valid_lossMSE)
            push!(validation_fes, valid_lossFE)
        end
    end
end

function pore(p, resolution; lowval=0.1)

    if resolution % 2==0
        inner = p/2*resolution ÷ 1
        w =  (p*resolution)/2 % 1

        isqb = Int((resolution/2+inner) ÷ 1) # square index to the right
        isqa = Int((resolution/2-inner) ÷ 1) # square index to the left
    else
        inner = p/2*resolution ÷ 1
        w = ((p*resolution)/2 % 1)
        w*= resolution ==1 ? 2 : 1
        isqb = Int((resolution/2+inner) ÷ 1) # square index to the right
        isqa = Int((resolution/2-inner) ÷ 1)+1 # square index to the left
    end

    begin
        A1 = ones((resolution, resolution))
        A2 = ones((resolution, resolution))
        A1[isqa+1:isqb, isqa+1:isqb] .= lowval

        if isqa==0 && w==0
            return A1
        elseif isqa==0 && w!=0
            return A1 .* (1-w)
        else
            A2[isqa:isqb+1, isqa:isqb+1] .= lowval
            return A1 .* (1-w) .+ A2 .* w
        end
    end

end

function generatepores(ps, resolution)
    n = sqrt(length(ps))
    N = resolution÷n
    @assert n == Int(n)
    @assert N == Int(N)
    n = Int(n)
    N = Int(N)
    pores = zeros((resolution, resolution))
    for (k,p) in enumerate(ps)
        i = (k-1)÷n
        j = (k-1)%n
        pores[i*N+1:(i+1)*N, j*N+1:(j+1)*N] = pore(p, N)
    end
    return pores
end

ChainRules.@non_differentiable generatepores(ps, resolution)

In [2]:
startp, endp = 7, 7
PEDSname = "PEDS"
equation = "exampleFisher"

ninit = 64
Jval = 1

cwval = 0.05
learningrate = 5e-5
nnodes =128

optimizerfunc, optname = Flux.ADAM, "ADAM"

number_epoch = 200
batchsize = 64

datalimitvalid = 2^10

Ks = Int[2^i for i=startp:endp]

coarseresolution = 4
Lx = Ly = 1
nin=Int(coarseresolution^2)
coarse_sim = setsimulation(Lx, Ly, coarseresolution);
nout = coarseresolution^2
const avgOpcoarse = create_cOp(coarseresolution);

endname = "$(equation)_$(2^startp)to$(2^endp)_ne$(number_epoch)_lr$(learningrate)_bs$(batchsize)_opt$(optname)_hn$(nnodes)_coarse$(coarseresolution)_seed";

##loading data
X = readdlm("../data/X_fisher$(coarseresolution^2)_small.csv", ',')
y = readdlm("../data/y_fisher$(coarseresolution^2)_small.csv", ',')[:];

Xv = X[:, 1:1024] #valid set
Xtest = X[:, end-1023:end] #test set
Xt = X[:, 1025:end]

yv = y[1:1024] #valid set
ytest = y[end-1023:end] #test set
yt = y[1025:end];

In [3]:
##Definition of problem constants
const debug = false
const drv = DataRunner(Xv, yv, [1]);
const al = ALstruct(J=Jval, Nvalid=datalimitvalid, batchsize=batchsize);
const valid = initvalid(al, drv) #validation loader

const drtest = DataRunner(Xtest, ytest, [1]);
const test = initvalid(al, drtest) #validation loader

##setup MPI and random
const comm = MPI.COMM_WORLD
const model_color = MPI.Comm_rank(comm)%al.J
const commModel = MPI.Comm_split(comm, model_color, 0)
const isleader = MPI.Comm_rank(commModel) == 0
const commLeader = MPI.Comm_split(comm, isleader, 0)
debug && print("Comm rank=$(MPI.Comm_rank(comm)), commModel rank = $(MPI.Comm_rank(commModel)), commLeader rank = $(MPI.Comm_rank(commLeader))\n")

Random.seed!(2138*(model_color+1)) #alter seed for different groups

function lossvalid(comm, commModel, commLeader, mloglik, m)
    valid_lossmean = 0.
    k = 0
    testmode!(m, true)
    for d in valid
        X,y=d
        valid_loss = dMSE(commModel, mloglik, m, X, y)

        valid_lossmean *= k
        k+=1
        valid_lossmean += valid_loss
        valid_lossmean /= k
    end
    testmode!(m, false)
    # MPI.Comm_rank(commModel)== 0 && @show valid_lossmean
    valid_lossmean = sum_reduce(commLeader, valid_lossmean) / al.J
    MPI.Comm_rank(comm) == 0 && @show valid_lossmean
    return valid_lossmean
end

function dFE_1d(comm, commModel, commLeader, m; valid=valid) """dFE computes the FE using parallelization over the batch with MPI"""
    evalsr = zeros(al.Nvalid)
    FE = 0.
    j=0
    ys = []
    testmode!(m, true)
    for (x, y) in valid
                rp = m(x[:,1+MPI.Comm_rank(commModel):MPI.Comm_size(commModel):length(y)])[:]
        evalsr[j*length(y)+1+MPI.Comm_rank(commModel):MPI.Comm_size(commModel):j*length(y)+length(y)] .= rp
        push!(ys, y...)
        j+=1
    end
    testmode!(m, false)
    evalsrModel = sum_reduce(commModel, evalsr)
    evalsr = sum_reduce(commLeader, evalsrModel) / al.J
    if MPI.Comm_rank(comm) == 0
        FE = norm(evalsr - ys)/norm(ys)
        MPI.Comm_rank(comm) == 0 && @show FE
    end
    return FE
end

dFE_1d (generic function with 1 method)

# PEDS

In [4]:
namemodel = "local_MSE_parallel_PEDSparameterized_$endname"

kval =  Ks[1]
##define same AL parameters for all workers
al1 = ALstruct(Ninit=ninit+8*kval, T=0, η=1e-3, J=Jval, ne=number_epoch, batchsize=batchsize);
if MPI.Comm_rank(comm) == 0
    @show kval
    @show 256+8*kval
end
##ititialize DataRunner and DataSet
dr = DataRunner(Xt, yt, [1]);
ds = DataSet()
validation_fes = []
mgen =  Chain(Dense(nin, nnodes, relu),
Dropout(0.5),
Dense(nnodes, nnodes,relu),
Dropout(0.5),
Dense(nnodes, coarseresolution^2, hardtanh),
x-> @. x*0.9/2 + 0.45 + .1)

cw=[cwval]

m(p) = begin
    generatedgeom = mgen(p)
    w = @. max(0, min(1, cw)) # harder function.
    @views map(i->Float32(targetfunc(avgOpcoarse * (w .* generatedgeom[:,i] + (1 .-w) .* generatepores(p[:, i],coarseresolution)[:]), sim=coarse_sim)), 1:length(p)÷ nin)
end
mloglik(p) = 1
ps = Flux.params(mgen, cw)


loader = initloader(al1, dr, ds);
opt = ADAM(al1.η)

if MPI.Comm_rank(comm)==0
    @time Flux.@epochs al1.ne train_distributed!(comm, commModel, commLeader, mloglik, m, dHuber, ps, loader, opt, validation_fes, logging=true)
else
    Flux.@epochs al1.ne train_distributed!(comm, commModel, commLeader, mloglik, m, dHuber, ps, loader, opt, validation_fes, logging=true)
end


# ##save models and validation FEs
# if isleader
#     BSON.@save "$(namemodel)_K$(kval)_mgen$(model_color).bson" mgen
#     BSON.@save "$(namemodel)_K$(kval)_cw$(model_color).bson" cw
# end

# if MPI.Comm_rank(comm) == 0
#     writedlm("$(namemodel)_K$(kval)_trainingMSE.csv", validation_fes, ',')
# end

# # evaluate and show the error on the test set
# valid_lossmean = dFE_1d(comm, commModel, commLeader, m)
# if MPI.Comm_rank(comm) == 0
#     @show valid_lossmean
#     writedlm("$(namemodel)_K$(kval)_endvalidFE.csv", [valid_lossmean], ',')
# end

kval = 128
256 + 8kval = 1280


┌ Info: Epoch 1
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 7.042909392808912e-5
valid_lossmean = 0.0061226879368676875
FE = 0.16059397858016888


┌ Info: Epoch 2
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 6.341972747794767e-5
valid_lossmean = 0.005388577354486344
FE = 0.15065906088131042


...

train_lossmean = 1.6818808751207033e-5
valid_lossmean = 0.0005073549984776466
FE = 0.04622896463115461


┌ Info: Epoch 199
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 1.6195499294460672e-5
valid_lossmean = 0.0005949161715017871
FE = 0.05005945099491915


┌ Info: Epoch 200
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 1.5092391685186291e-5
valid_lossmean = 0.0004181320856671735
FE = 0.041967678135964165
107.586316 seconds (372.97 M allocations: 42.558 GiB, 10.08% gc time, 1.05% compilation time)


In [5]:
dFE_1d(comm, commModel, commLeader, m, valid=test)

FE = 0.04814524309206283


0.04814524309206283

## PEDS result

The fractional error on the test set is 0.0481.

# Baseline 

In [6]:
namemodel = "local_MSE_parallel_baselineparameterized_$endname"

##define same AL parameters for all workers
al1 = ALstruct(Ninit=ninit+8*kval, T=0, η=1e-3, J=Jval, ne=number_epoch, batchsize=batchsize);
if MPI.Comm_rank(comm) == 0
    @show kval
    @show 256+8*kval
end
##ititialize DataRunner and DataSet
dr = DataRunner(Xt, yt, [1]);
# dr = DataRunner(generateX(Xt), yt, [1]);
ds = DataSet()
validation_fes = []
##initialize baseline
mgen =  Chain(Dense(nin, nnodes, relu),
Dropout(0.5),
#BatchNorm(256, relu),
Dense(nnodes, nnodes, relu),
Dropout(0.5),
#BatchNorm(256, relu),
Dense(nnodes, coarseresolution^2, relu),
#BatchNorm(coarseresolution^2, hardtanh),
Dense(coarseresolution^2, 1, hardtanh),
x-> @. x[:]*0.9/2 + 0.45 + .1)

cw=[cwval]

m(p) = begin
    mgen(p)
end
mloglik(p) = 1
ps = Flux.params(mgen)


loader = initloader(al1, dr, ds);
opt = ADAM(al1.η)

if MPI.Comm_rank(comm)==0
    @time Flux.@epochs al1.ne train_distributed!(comm, commModel, commLeader, mloglik, m, dHuber, ps, loader, opt, validation_fes, logging=true)
else
    Flux.@epochs al1.ne train_distributed!(comm, commModel, commLeader, mloglik, m, dHuber, ps, loader, opt, validation_fes, logging=true)
end


# ##save models and validation FEs
# if isleader
#     BSON.@save "$(namemodel)_K$(kval)_mgen$(model_color).bson" mgen
# end

# if MPI.Comm_rank(comm) == 0
#     writedlm("$(namemodel)_K$(kval)_trainingMSE.csv", validation_fes, ',')
# end

# # evaluate and show the error on the test set
# valid_lossmean = dFE_1d(comm, commModel, commLeader, m)
# if MPI.Comm_rank(comm) == 0
#     @show valid_lossmean
#     writedlm("$(namemodel)_K$(kval)_endvalidFE.csv", [valid_lossmean], ',')
# end



kval = 128
256 + 8kval = 1280


┌ Info: Epoch 1
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 9.087440830678339e-5
valid_lossmean = 0.007655771921302229
FE = 0.17957781724323424
train_lossmean = 7.882662053470223e-5
valid_lossmean = 0.009459196166458565
FE = 0.19961138645332263


┌ Info: Epoch 2
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154
┌ Info: Epoch 3
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 7.351658493128562e-5
valid_lossmean = 0.00813197924014964
FE = 0.18507865118120526


...

train_lossmean = 2.1766606287856584e-5
valid_lossmean = 0.003759685576469673
FE = 0.12584441849140598


┌ Info: Epoch 198
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 2.1700764215927183e-5
valid_lossmean = 0.004076829577912635
FE = 0.13104470386333522
train_lossmean = 2.20650755241679e-5
valid_lossmean = 0.003203319350285856
FE = 0.11616045747354867


┌ Info: Epoch 199
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154
┌ Info: Epoch 200
└ @ Main /Users/raphaelpestourie/.julia/packages/Flux/EXOFx/src/optimise/train.jl:154


train_lossmean = 2.1464521044725547e-5
valid_lossmean = 0.0036031543992685916
FE = 0.12319685789939004
 18.248528 seconds (7.97 M allocations: 9.089 GiB, 9.29% gc time, 9.73% compilation time)


In [7]:
dFE_1d(comm, commModel, commLeader, m, valid=test)

FE = 0.13068358691983506


0.13068358691983506

## Baseline result 

The fractional error on the test set is 0.13.

# Overall result

With about 1000 data points, a single model PEDS model achieves an error < 5% and leads to a 2.7x improvement compared to the NN-only baseline. 